# Непрерывные счётчики времени

## Определения времён

Wall time - разница между временами конца и начала выполнения программы.

CPU time (process time) - время, в течение которого CPU выполнял программу.

User time - время, в течение которого CPU выполнял программу в user space.

System time - время, в течение которого CPU выполнял программу в kernel space.

## timeit

Модуль применяется для бенчмаркинга.

Пример

In [128]:
import timeit
timeit.timeit(stmt='"g" in text', setup='text = "sample string"')

0.0604657279982348

Основные функции

In [20]:
default_timer = time.perf_counter

timeit(stmt='pass', setup='pass', timer=default_timer, number=1000000, globals=None) 
# вызывает setup 1 раз, вызывает stmt number раз, возвращает среднее время
# перед замером времени выключается сборщик мусора

repeat(stmt='pass', setup='pass', timer=default_timer, repeat=5, number=1000000, globals=None) 
# вызывает timeit repeat раз, возвращает лист времён

T = Timer(stmt="pass", setup="pass", timer=default_timer, globals=None)
T.autorange(callback=None)
# вызывает timeit с возрастающими значениями number пока замеряемое время < 0.2 секунд

Вычисление времени исполнения производится с помощью параметра timer, значение по умолчанию - perf_counter из модуля time.

## time

In [267]:
import time

time.sleep(1)

Задерживает исполнение вызывающего треда на переданное число секунд. Если получен сигнал и обработчик сигнала бросает исключение задержка может длится меньше указанного времени.

In [268]:
import warnings
warnings.filterwarnings('ignore')

time_0 = time.clock()
time.sleep(1)
time_1 = time.clock()
time_1 - time_0

0.003106999999999971

Unix : возвращает current processor time в секундах с момента начала процесса или c момента первого вызова clock(). Поведение и точность зависят от платформы.

Windows: возвращает wall time в секундах с момента первого вызова clock(). Точность обычно лучше 1 микросекунды.

Deprecated.

In [186]:
time.pthread_getcpuclockid(thread_id) # возвращает clk_id счётчика, который возвращает CPU time для thread c переданным thread_id

In [269]:
possible_clk_id_constants = ['CLOCK_BOOTTIME', 'CLOCK_HIGHRES', 'CLOCK_MONOTONIC', 'CLOCK_MONOTONIC_RAW', 'CLOCK_PROCESS_CPUTIME_ID', 'CLOCK_PROF', \
                    'CLOCK_THREAD_CPUTIME_ID', 'CLOCK_UPTIME', 'CLOCK_REALTIME']
for clk_id_constant in possible_clk_id_constants:
    if hasattr(time, clk_id_constant):
        print(clk_id_constant, ':', time.clock_getres(getattr(time, clk_id_constant)))

CLOCK_MONOTONIC : 1.0000000000000002e-06
CLOCK_MONOTONIC_RAW : 1e-09
CLOCK_PROCESS_CPUTIME_ID : 1.0000000000000002e-06
CLOCK_THREAD_CPUTIME_ID : 1e-09
CLOCK_REALTIME : 1.0000000000000002e-06


In [ ]:
time.clock_gettime(clk_id) → float
time.clock_gettime_ns(clk_id) → int
time.clock_settime(clk_id, time: float)
time.clock_settime_ns(clk_id, time: int)

In [270]:
for name in ['clock', 'monotonic', 'perf_counter', 'process_time', 'thread_time', 'time']:
    print(name, ':', time.get_clock_info(name))

clock : namespace(adjustable=False, implementation='clock()', monotonic=True, resolution=1e-06)
monotonic : namespace(adjustable=False, implementation='mach_absolute_time()', monotonic=True, resolution=1e-09)
perf_counter : namespace(adjustable=False, implementation='mach_absolute_time()', monotonic=True, resolution=1e-09)
process_time : namespace(adjustable=False, implementation='clock_gettime(CLOCK_PROCESS_CPUTIME_ID)', monotonic=True, resolution=1.0000000000000002e-06)
thread_time : namespace(adjustable=False, implementation='clock_gettime(CLOCK_THREAD_CPUTIME_ID)', monotonic=True, resolution=1e-09)
time : namespace(adjustable=True, implementation='clock_gettime(CLOCK_REALTIME)', monotonic=False, resolution=1.0000000000000002e-06)


adjustable = True, если счётчик может быть изменён.

In [271]:
time.clock_settime(time.CLOCK_THREAD_CPUTIME_ID, 0)

OSError: [Errno 22] Invalid argument

In [272]:
time.clock_settime(time.CLOCK_REALTIME, 0)

PermissionError: [Errno 1] Operation not permitted

In [273]:
time_0 = time.monotonic()
time.sleep(1)
time_1 = time.monotonic()
time_1 - time_0

1.00098709899612

Возвращает монотонное wall time время в секундах с неопределённого момента. Значение не зависит от изменений системных часов.

In [289]:
time_0 = time.perf_counter()
time.sleep(1)
time_1 = time.perf_counter()
time_1 - time_0

1.0054481449988089

Возвращает wall time время счётчика с наибольшей точностью в секундах с неопределённого момента.

In [276]:
time_0 = time.process_time()
time.sleep(1)
time_1 = time.process_time()
time_1 - time_0

0.0045229999999989445

Возвращает сумму system time и user time текущего процесса в секундах с неопределённого момента.

In [286]:
time_0 = time.thread_time()
time.sleep(1)
time_1 = time.thread_time()
time_1 - time_0

0.0003354079999979831

Аналог time.process_time() для тредов.

In [280]:
time.time()

1552406099.140644

Возвращает wall time с эпохи в секундах. Обычно эпоха = 01.01.1970 00:00:00 (UTC), также эпоху возвращает gmtime(0).

Если между вызовами системные часы были изменены, то возвращаемые значения могут быть немонотонными.

In [237]:
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

## asyncio.loop.time()

In [283]:
# base_event.py

class BaseEventLoop(events.AbstractEventLoop):
  def time(self):
          """Return the time according to the event loop's clock.

          This is a float expressed in seconds since an epoch, but the
          epoch, precision, accuracy and drift are unspecified and may
          differ per event loop.
          """
          return time.monotonic()

In [281]:
import asyncio

loop = asyncio.new_event_loop()

for i in range(5):
  await asyncio.sleep(1)
  print(loop.time())

loop.stop()
loop.close()

36628.053519912
36629.05905599
36630.062621697
36631.063375145
36632.067191352
